In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

# Preparamos los Sets de Training

In [2]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets"

In [3]:
ins_18_20 = pd.read_csv( loc_ts + "\\ins_18_20.csv" ); 
ins_21_23 = pd.read_csv( loc_ts + "\\ins_21_23.csv" ); 
ins_24_26 = pd.read_csv( loc_ts + "\\ins_24_26.csv" ); 

In [5]:
auc_18_20 = pd.read_csv( loc_ts + "\\auc_18_20.csv" ); 
auc_21_23 = pd.read_csv( loc_ts + "\\auc_21_23.csv" );
auc_24_26 = pd.read_csv( loc_ts + "\\auc_24_26.csv" );

# Genero los Sets con el primer install

In [7]:
auc_18_20["date"] =  pd.to_datetime( auc_18_20["date"] ); auc_18_20["_st"] = auc_18_20["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); auc_18_20["_st"] = auc_18_20["_st"].dt.total_seconds();
auc_21_23["date"] =  pd.to_datetime( auc_21_23["date"] ); auc_21_23["_st"] = auc_21_23["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); auc_21_23["_st"] = auc_21_23["_st"].dt.total_seconds(); 
auc_24_26["date"] =  pd.to_datetime( auc_24_26["date"] ); auc_24_26["_st"] = auc_24_26["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); auc_24_26["_st"] = auc_24_26["_st"].dt.total_seconds(); 

In [8]:
auc_18_20_first_auc = auc_18_20.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )
auc_21_23_first_auc = auc_21_23.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )
auc_24_26_first_auc = auc_24_26.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )

# Sacamos los Features

In [9]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Features\ftr_auc"

In [10]:
rh_18_20 = auc_18_20_first_auc[ ["ref_hash"] ] #RH unicos que coinciden con los datos que usamos para entrenar
rh_21_23 = auc_21_23_first_auc[ ["ref_hash"] ] #RH unicos que coinciden con los datos que queremos predecir

tg_18_20 = auc_18_20_first_auc[ ["_st"] ]
tg_21_23 = auc_21_23_first_auc[ ["_st"] ]

## <span style="color:green"> **Tiempo hasta la primer instlacion en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer instlacion

In [ ]:
frst_ins_18_20 = rh_18_20.copy()
frst_ins_21_23 = rh_21_23.copy()

first_installs_rh_auc_18_20 = ins_18_20[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_installs_rh_auc_21_23 = ins_21_23[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_installs_rh_auc_18_20["time_to_frt_ins"] = ( pd.to_datetime( first_installs_rh_auc_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_installs_rh_auc_21_23["time_to_frt_ins"] = ( pd.to_datetime( first_installs_rh_auc_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_installs_rh_auc_18_20.drop( "date", axis = 1, inplace = True )
first_installs_rh_auc_21_23.drop( "date", axis = 1, inplace = True )

frst_ins_18_20 = frst_ins_18_20.merge( first_installs_rh_auc_18_20, how = "left", on = "ref_hash" )
frst_ins_21_23 = frst_ins_21_23.merge( first_installs_rh_auc_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_ins_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_ins_21_23.fillna( 3 * 24 * 3600, inplace = True )

frst_ins_18_20.to_csv( loc_ftr + "\\frst_ins_18_20.csv", index = False )
frst_ins_21_23.to_csv( loc_ftr + "\\frst_ins_21_23.csv", index = False )

## <span style="color:green"> **Aplicacion mas instalada por el usuario** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la app principal de algun dispositivo)

In [ ]:
main_app_18_20 = rh_18_20.copy()
main_app_21_23 = rh_21_23.copy()

app_mas_ins_18_20 = ins_18_20.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()
app_mas_ins_21_23 = ins_21_23.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()

app_mas_ins_18_20 = app_mas_ins_18_20.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_18_20["cant_ins"] )
app_mas_ins_21_23 = app_mas_ins_21_23.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_21_23["cant_ins"] ) 

main_app_18_20 = main_app_18_20.merge( app_mas_ins_18_20, how = "left", on = "ref_hash" )
main_app_21_23 = main_app_21_23.merge( app_mas_ins_21_23, how = "left", on = "ref_hash" )

main_app_18_20["to_count"] = 1; main_app_18_20["application_id"] = main_app_18_20[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_18_20); del(main_app_18_20["to_count"])
main_app_21_23["to_count"] = 1; main_app_21_23["application_id"] = main_app_21_23[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_21_23); del(main_app_21_23["to_count"])

main_app_21_23["application_id"].fillna( main_app_21_23["application_id"].isnull().sum() / len(main_app_21_23), inplace = True )
main_app_18_20["application_id"].fillna( main_app_18_20["application_id"].isnull().sum() / len(main_app_18_20), inplace = True )

main_app_18_20.to_csv( loc_ftr + "\\main_app_18_20.csv", index = False )
main_app_21_23.to_csv( loc_ftr + "\\main_app_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones atribuidas por dispositivo** </span> 

In [ ]:
cins_atr_18_20 = rh_18_20.copy()
cins_atr_21_23 = rh_21_23.copy()

cant_atr_ins_18_20 = ins_18_20[["ref_hash", "attributed"]]; cant_atr_ins_18_20["attributed"] = cant_atr_ins_18_20["attributed"].apply( lambda x: 1 if x else 0 );
cant_atr_ins_21_23 = ins_21_23[["ref_hash", "attributed"]]; cant_atr_ins_21_23["attributed"] = cant_atr_ins_21_23["attributed"].apply( lambda x: 1 if x else 0 );

cant_atr_ins_18_20 = cant_atr_ins_18_20.groupby( "ref_hash" ).agg( "sum" )
cant_atr_ins_21_23 = cant_atr_ins_21_23.groupby( "ref_hash" ).agg( "sum" )

cins_atr_18_20 = cins_atr_18_20.merge( cant_atr_ins_18_20, how = "left", on = "ref_hash" ).fillna( 0 )
cins_atr_21_23 = cins_atr_21_23.merge( cant_atr_ins_21_23, how = "left", on = "ref_hash" ).fillna( 0 )

cins_atr_18_20.to_csv( loc_ftr + "\\cins_atr_18_20.csv", index = False )
cins_atr_21_23.to_csv( loc_ftr + "\\cins_atr_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones implicitas por dispositivo** </span> 

In [ ]:
cins_imp_18_20 = rh_18_20.copy()
cins_imp_21_23 = rh_21_23.copy()

cant_imp_ins_18_20 = ins_18_20[["ref_hash", "implicit"]]; cant_imp_ins_18_20["implicit"] = cant_imp_ins_18_20["implicit"].apply( lambda x: 1 if x else 0 );
cant_imp_ins_21_23 = ins_21_23[["ref_hash", "implicit"]]; cant_imp_ins_21_23["implicit"] = cant_imp_ins_21_23["implicit"].apply( lambda x: 1 if x else 0 );

cant_imp_ins_18_20 = cant_imp_ins_18_20.groupby( "ref_hash" ).agg( "sum" )
cant_imp_ins_21_23 = cant_imp_ins_21_23.groupby( "ref_hash" ).agg( "sum" )

cins_imp_18_20 = cins_imp_18_20.merge( cant_imp_ins_18_20, how = "left", on = "ref_hash" ).fillna( 0 )
cins_imp_21_23 = cins_imp_21_23.merge( cant_imp_ins_21_23, how = "left", on = "ref_hash" ).fillna( 0 )

cins_imp_18_20.to_csv( loc_ftr + "\\cins_imp_18_20.csv", index = False )
cins_imp_21_23.to_csv( loc_ftr + "\\cins_imp_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta la ultima instalacion en esa ventana** </span>

In [41]:
last_ins_18_20 = rh_18_20.copy()
last_ins_21_23 = rh_21_23.copy()

last_installs_rh_auc_18_20 = ins_18_20[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )
last_installs_rh_auc_21_23 = ins_21_23[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )

last_installs_rh_auc_18_20["time_to_lst_ins"] = ( pd.to_datetime( last_installs_rh_auc_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_installs_rh_auc_21_23["time_to_lst_ins"] = ( pd.to_datetime( last_installs_rh_auc_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_installs_rh_auc_18_20.drop( "date", axis = 1, inplace = True )
last_installs_rh_auc_21_23.drop( "date", axis = 1, inplace = True )

last_ins_18_20 = last_ins_18_20.merge( last_installs_rh_auc_18_20, how = "left", on = "ref_hash" )
last_ins_21_23 = last_ins_21_23.merge( last_installs_rh_auc_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_ins_18_20.fillna( 3 * 24 * 3600, inplace = True )
last_ins_21_23.fillna( 3 * 24 * 3600, inplace = True )

last_ins_18_20.to_csv( loc_ftr + "\\last_ins_18_20.csv", index = False )
last_ins_21_23.to_csv( loc_ftr + "\\last_ins_21_23.csv", index = False )

## <span style="color:green"> **Instalo entre 21 hs y 3 hs (Noche)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [46]:
ins_18_20['ins_21_3'] = ( pd.to_datetime( ins_18_20['date'] ).dt.hour > 20) | ( pd.to_datetime( ins_18_20['date'] ).dt.hour < 4) 
ins_21_23['ins_21_3'] = ( pd.to_datetime( ins_21_23['date'] ).dt.hour > 20) | ( pd.to_datetime( ins_21_23['date'] ).dt.hour < 4)

ins_night_18_20 = rh_18_20.copy()
ins_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()

ins_night_18_20 = ins_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_night_21_23 = ins_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_night_18_20["ins_21_3"] =  (ins_night_18_20["ins_21_3"] > 1).astype('int8')
ins_night_21_23["ins_21_3"] =  (ins_night_21_23["ins_21_3"] > 1).astype('int8')

ins_night_18_20['me_ins_21_3'] = ( ins_night_18_20.groupby('ins_21_3').transform('count') ) / len(ins_night_18_20['ins_21_3'])
ins_night_21_23['me_ins_21_3'] = ( ins_night_21_23.groupby('ins_21_3').transform('count') ) / len(ins_night_21_23['ins_21_3'])

del ins_night_18_20["ins_21_3"]
del ins_night_21_23["ins_21_3"]

ins_night_18_20.to_csv( loc_ftr + "\\ins_ngme_18_20.csv", index = False )
ins_night_21_23.to_csv( loc_ftr + "\\ins_ngme_21_23.csv", index = False )

del ins_18_20['ins_21_3']
del ins_21_23['ins_21_3']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [47]:
ins_18_20['ins_21_3'] = ( pd.to_datetime( ins_18_20['date'] ).dt.hour > 20) | ( pd.to_datetime( ins_18_20['date'] ).dt.hour < 4) 
ins_21_23['ins_21_3'] = ( pd.to_datetime( ins_21_23['date'] ).dt.hour > 20) | ( pd.to_datetime( ins_21_23['date'] ).dt.hour < 4)

ins_night_18_20 = rh_18_20.copy()
ins_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()

ins_night_18_20 = ins_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_night_21_23 = ins_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_night_18_20["ins_21_3"] =  (ins_night_18_20["ins_21_3"] > 1).astype('int8')
ins_night_21_23["ins_21_3"] =  (ins_night_21_23["ins_21_3"] > 1).astype('int8')

ins_night_18_20.to_csv( loc_ftr + "\\ins_ngoh_18_20.csv", index = False )
ins_night_21_23.to_csv( loc_ftr + "\\ins_ngoh_21_23.csv", index = False )

del ins_18_20['ins_21_3']
del ins_21_23['ins_21_3']

## <span style="color:green"> **Instalo entre 4 hs y 10 hs (Mañana)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [48]:
ins_18_20['ins_4_10'] = ( pd.to_datetime( ins_18_20['date'] ).dt.hour > 3) & ( pd.to_datetime( ins_18_20['date'] ).dt.hour < 11) 
ins_21_23['ins_4_10'] = ( pd.to_datetime( ins_21_23['date'] ).dt.hour > 3) & ( pd.to_datetime( ins_21_23['date'] ).dt.hour < 11)

ins_morn_18_20 = rh_18_20.copy()
ins_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()

ins_morn_18_20 = ins_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_morn_21_23 = ins_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_morn_18_20["ins_4_10"] =  (ins_morn_18_20["ins_4_10"] > 1).astype('int8')
ins_morn_21_23["ins_4_10"] =  (ins_morn_21_23["ins_4_10"] > 1).astype('int8')

ins_morn_18_20['me_ins_4_10'] = ( ins_morn_18_20.groupby('ins_4_10').transform('count') ) / len(ins_morn_18_20['ins_4_10'])
ins_morn_21_23['me_ins_4_10'] = ( ins_morn_21_23.groupby('ins_4_10').transform('count') ) / len(ins_morn_21_23['ins_4_10'])

del ins_morn_18_20["ins_4_10"]
del ins_morn_21_23["ins_4_10"]

ins_morn_18_20.to_csv( loc_ftr + "\\ins_mrme_18_20.csv", index = False )
ins_morn_21_23.to_csv( loc_ftr + "\\ins_mrme_21_23.csv", index = False )

del ins_18_20['ins_4_10']
del ins_21_23['ins_4_10']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [49]:
ins_18_20['ins_4_10'] = ( pd.to_datetime( ins_18_20['date'] ).dt.hour > 3) & ( pd.to_datetime( ins_18_20['date'] ).dt.hour< 11) 
ins_21_23['ins_4_10'] = ( pd.to_datetime( ins_21_23['date'] ).dt.hour > 3) & ( pd.to_datetime( ins_21_23['date'] ).dt.hour < 11)

ins_morn_18_20 = rh_18_20.copy()
ins_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()

ins_morn_18_20 = ins_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_morn_21_23 = ins_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_morn_18_20["ins_4_10"] =  (ins_morn_18_20["ins_4_10"] > 1).astype('int8')
ins_morn_21_23["ins_4_10"] =  (ins_morn_21_23["ins_4_10"] > 1).astype('int8')

ins_morn_18_20.to_csv( loc_ftr + "\\ins_mroh_18_20.csv", index = False )
ins_morn_21_23.to_csv( loc_ftr + "\\ins_mroh_21_23.csv", index = False )

del ins_18_20['ins_4_10']
del ins_21_23['ins_4_10']

## <span style="color:green"> **Instalo entre 11 hs y 15 hs (Medio dia)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [51]:
ins_18_20['ins_11_15'] = (  pd.to_datetime( ins_18_20['date'] ).dt.hour > 10) & ( pd.to_datetime( ins_18_20['date'] ).dt.hour < 16) 
ins_21_23['ins_11_15'] = (  pd.to_datetime( ins_21_23['date'] ).dt.hour > 10) & ( pd.to_datetime( ins_21_23['date'] ).dt.hour < 16)

ins_midday_18_20 = rh_18_20.copy()
ins_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()

ins_midday_18_20 = ins_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_midday_21_23 = ins_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_midday_18_20["ins_11_15"] =  (ins_midday_18_20["ins_11_15"] > 1).astype('int8')
ins_midday_21_23["ins_11_15"] =  (ins_midday_21_23["ins_11_15"] > 1).astype('int8')

ins_midday_18_20['me_ins_11_15'] = ( ins_midday_18_20.groupby('ins_11_15').transform('count') ) / len(ins_midday_18_20['ins_11_15'])
ins_midday_21_23['me_ins_11_15'] = ( ins_midday_21_23.groupby('ins_11_15').transform('count') ) / len(ins_midday_21_23['ins_11_15'])

del ins_midday_18_20["ins_11_15"]
del ins_midday_21_23["ins_11_15"]

ins_midday_18_20.to_csv( loc_ftr + "\\ins_mdme_18_20.csv", index = False )
ins_midday_21_23.to_csv( loc_ftr + "\\ins_mdme_21_23.csv", index = False )

del ins_18_20['ins_11_15']
del ins_21_23['ins_11_15']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [53]:
ins_18_20['ins_11_15'] = ( pd.to_datetime( ins_18_20['date'] ).dt.hour > 10) & ( pd.to_datetime( ins_18_20['date'] ).dt.hour < 16) 
ins_21_23['ins_11_15'] = (pd.to_datetime( ins_18_20['date'] ).dt.hour > 10) & ( pd.to_datetime( ins_18_20['date'] ).dt.hour < 16)

ins_midday_18_20 = rh_18_20.copy()
ins_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()

ins_midday_18_20 = ins_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_midday_21_23 = ins_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_midday_18_20["ins_11_15"] =  (ins_midday_18_20["ins_11_15"] > 1).astype('int8')
ins_midday_21_23["ins_11_15"] =  (ins_midday_21_23["ins_11_15"] > 1).astype('int8')

ins_midday_18_20.to_csv( loc_ftr + "\\ins_mdoh_18_20.csv", index = False )
ins_midday_21_23.to_csv( loc_ftr + "\\ins_mdoh_21_23.csv", index = False )

del ins_18_20['ins_11_15']
del ins_21_23['ins_11_15']

## <span style="color:green"> **Instalo entre 16 hs y 20 hs (Tarde)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [54]:
ins_18_20['ins_16_20'] = ( pd.to_datetime( ins_18_20['date'] ).dt.hour > 15) & ( pd.to_datetime( ins_18_20['date'] ).dt.hour < 21) 
ins_21_23['ins_16_20'] = ( pd.to_datetime( ins_21_23['date'] ).dt.hour > 15) & ( pd.to_datetime( ins_21_23['date'] ).dt.hour < 21)

ins_after_18_20 = rh_18_20.copy()
ins_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()

ins_after_18_20 = ins_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_after_21_23 = ins_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_after_18_20["ins_16_20"] =  (ins_after_18_20["ins_16_20"] > 1).astype('int8')
ins_after_21_23["ins_16_20"] =  (ins_after_21_23["ins_16_20"] > 1).astype('int8')

ins_after_18_20['me_ins_16_20'] = ( ins_after_18_20.groupby('ins_16_20').transform('count') ) / len(ins_after_18_20['ins_16_20'])
ins_after_21_23['me_ins_16_20'] = ( ins_after_21_23.groupby('ins_16_20').transform('count') ) / len(ins_after_21_23['ins_16_20'])

del ins_after_18_20["ins_16_20"]
del ins_after_21_23["ins_16_20"]


ins_after_18_20.to_csv( loc_ftr + "\\ins_afoh_18_20.csv", index = False )
ins_after_21_23.to_csv( loc_ftr + "\\ins_afoh_21_23.csv", index = False )

del ins_18_20['ins_16_20']
del ins_21_23['ins_16_20']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [55]:
ins_18_20['ins_16_20'] = ( pd.to_datetime( ins_18_20['date'] ).dt.hour > 15) & ( pd.to_datetime( ins_18_20['date'] ).dt.hour < 21) 
ins_21_23['ins_16_20'] = ( pd.to_datetime( ins_21_23['date'] ).dt.hour > 15) & ( pd.to_datetime( ins_21_23['date'] ).dt.hour < 21)

ins_after_18_20 = rh_18_20.copy()
ins_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()

ins_after_18_20 = ins_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_after_21_23 = ins_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_after_18_20["ins_16_20"] =  (ins_after_18_20["ins_16_20"] > 1).astype('int8')
ins_after_21_23["ins_16_20"] =  (ins_after_21_23["ins_16_20"] > 1).astype('int8')

ins_after_18_20.to_csv( loc_ftr + "\\ins_afme_18_20.csv", index = False )
ins_after_21_23.to_csv( loc_ftr + "\\ins_afme_21_23.csv", index = False )

del ins_18_20['ins_16_20']
del ins_21_23['ins_16_20']